In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from Evaluation import get_metrics

---
## Read Data

In [5]:
# path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'
# path = '../'

In [21]:
data_path = 'Data/Amazon/'
file_name = 'am_40k_users'
data_path = 'Data/ML/'
file_name = 'ml_1m'

In [22]:
df = pd.read_pickle(path + data_path + file_name)
total_items = df.item_id.nunique()
total_users = df.user_id.nunique()

In [23]:
train_set = pd.read_pickle(path + data_path + file_name + '_train')
# val_set = pd.read_pickle(path + data_path + file_name + '_val')
test_set = pd.read_pickle(path + data_path + file_name + '_test')

---
# Set Params

In [27]:
# Eval
steps = 5
rank_at = 20

In [24]:
bpr_params = {
"nolf": 8, #Size of latent feature vectors
"n_iterations": 20, #around 20 is sufficient
"sample_size": 0.5*len(train_set),
"seed": 1234,
"alpha": 0.08, # Learning rate: Amazon_01 best: 0.08
"rho": 1.05, # Bold driver increase alpha
"sigma": 0.5, # Bold driver decrease alpha
          
"reg_user": 0, #0.1, # Am_40k best: 0.1 # ML_1 best: 0
"reg_item": 0, #0.1, # Am_40k best: 0.1 # ML_1 best: 0
}

In [25]:
cfrnn_params = {
'train_time':0,
'epochs':10,
'BATCH_SIZE':32,
'learning_rate':0.1,
'delta':0.01,             # Diversity Bias
'max_seq_len':30,        # Max length of sequence71=median
'embedding_dim':100,
'rnn_units':20,
'ckpt_dir': '../ckpts/ckpts',

'test_users': 500,
'val_users': 500,

'pad_value':total_items, # Pad with total_items+1 => masked => still use item 0
'shift_targets_by':1     
}

---
# BPR

In [26]:
from BPR import BPR
bpr = BPR(total_users, total_items, bpr_params)

In [ ]:
samples_name = data_path + '/samples/' + file_name + '_samples'

In [ ]:
%%time
bpr.train_model(sample_path=path + samples_name + '.npy', val_set=val_set)

In [ ]:
bpr.store_model(path + '/logs/' + file_name + '_log')
bpr.plot_training()

In [ ]:
ranked_df_bpr = bpr.get_predictions(test_set, stats=False)
bpr_metrics = get_metrics(ranked_df_bpr, steps, rank_at, stats=False)
bpr_metrics

In [ ]:
bpr_metrics.to_pickle(path + 'Results/' + file_name + '_bpr_metrics')

---
# CFRNN

In [ ]:
from CFRNN import CFRNN
cfrnn = CFRNN(total_users, total_items, cfrnn_params)
cfrnn.build_model(summary=False)
cfrnn.compile_model(train_set=train_set)

In [ ]:
train_dataset = cfrnn.create_seq_batch_tf_dataset(train_set, stats=False)
val_dataset = cfrnn.create_seq_batch_tf_dataset(val_set, stats=False)

In [ ]:
cfrnn.train(train_dataset, val_dataset, verbose=1)
cfrnn.plot_training

In [ ]:
ranked_df_cfrnn = cfrnn.get_predictions(test_set, test_leave_one_out, batch_size=1024, rank_at=rank_at, ckpt_dir=cfrnn_params['ckpt_dir'])
cfrnn_metrics = get_metrics(ranked_df_cfrnn, steps, rank_at, stats=False)
cfrnn_metrics

---
# NCF